### Required Discussion 19:1: Building a Recommender System with SURPRISE

This discussion focuses on exploring additional algorithms with the `Suprise` library to generate recommendations.  Your goal is to identify the optimal algorithm by minimizing the mean squared error using cross validation. You are also going to select a dataset to use from [grouplens](https://grouplens.org/datasets/movielens/) example datasets.  

To begin, head over to [grouplens](https://grouplens.org/datasets/movielens/) and examine the different datasets available.  Choose one so that it is easy to create the data as expected in `Surprise` with user, item, and rating information.  Then, compare the performance of at least the `KNNBasic`, `SVD`, `NMF`, `SlopeOne`, and `CoClustering` algorithms to build your recommendations.  For more information on the algorithms see the documentation for the algorithm package [here](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html).

Share the results of your investigation and include the results of your cross validation and a basic description of your dataset with your peers.


In [ ]:
# Install SURPRISE (run this only once)
# !pip install scikit-surprise


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, NMF, KNNBasic, SlopeOne, CoClustering
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the ratings.csv file
ratings_full_df = pd.read_csv('data/ratings.csv')

# Sample a smaller subset (e.g., 10% of ratings)
ratings_df = ratings_full_df.sample(frac=0.1, random_state=42)

# Display the first few rows to verify the data
print(ratings_df.head())


          userId  movieId  rating   timestamp
10685861   66954      781     5.0   850944577
1552723     9877      574     4.0   945495614
6145184    38348     1088     2.0   999974867
16268584  101952     2706     1.0  1203077565
22418634  140400   275079     3.5  1653782463


In [14]:
# Summary statistics for the ratings
print("\nRatings summary:")
print(ratings_df['rating'].describe())


Ratings summary:
count    3.200020e+06
mean     3.540462e+00
std      1.058937e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64


In [15]:
# Count unique users and movies
n_users = ratings_df['userId'].nunique()
n_movies = ratings_df['movieId'].nunique()
n_ratings = len(ratings_df)
print(f"\nDataset has {n_users} users, {n_movies} movies, and {n_ratings} ratings")


Dataset has 197270 users, 42809 movies, and 3200020 ratings


In [3]:
# Create a Reader object
reader = Reader(rating_scale=(0.5, 5.0))

In [16]:
# Load the data into SURPRISE format
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# Define the algorithms to compare ( I had to do one at a time due to my laptop's performance issues...)
algorithms = {
    'SVD': SVD(),
    # 'NMF': NMF(),
    # 'KNNBasic': KNNBasic(),
    # 'SlopeOne': SlopeOne(),
    # 'CoClustering': CoClustering()
}

# Dictionary to store the results
results = {}

# Perform cross-validation for each algorithm
for name, algorithm in algorithms.items():
    print(f"\nEvaluating {name}...")
    cv_results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    results[name] = {
        'test_rmse': cv_results['test_rmse'].mean(),
        'test_mae': cv_results['test_mae'].mean(),
        'fit_time': cv_results['fit_time'].mean(),
        'test_time': cv_results['test_time'].mean()
    }


Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8956  0.8963  0.8965  0.8966  0.8973  0.8965  0.0005  
MAE (testset)     0.6846  0.6845  0.6853  0.6855  0.6857  0.6851  0.0005  
Fit time          41.77   42.60   40.54   44.60   40.41   41.98   1.54    
Test time         5.09    6.02    6.73    4.97    6.09    5.78    0.66    


AttributeError: 'tuple' object has no attribute 'mean'

# SVD Algorithm Results

| Metric | Fold 1 | Fold 2 | Fold 3 | Fold 4 | Fold 5 | Mean | Std |
|--------|--------|--------|--------|--------|--------|------|------|
| RMSE (testset) | 0.8956 | 0.8963 | 0.8965 | 0.8966 | 0.8973 | 0.8965 | 0.0005 |
| MAE (testset) | 0.6846 | 0.6845 | 0.6853 | 0.6855 | 0.6857 | 0.6851 | 0.0005 |
| Fit time | 41.77 | 42.60 | 40.54 | 44.60 | 40.41 | 41.98 | 1.54 |
| Test time | 5.09 | 6.02 | 6.73 | 4.97 | 6.09 | 5.78 | 0.66 |

These are solid results for SVD. An RMSE of about 0.897 means your predictions are, on average, within about 0.9 stars of the actual ratings users gave. The consistency across folds (low standard deviation of 0.001) suggests the model is stable. 0.12 |

In [8]:
# Perform cross-validation for NMF algorithm
name = 'NMF'
algorithm = NMF()
print(f"\nEvaluating {name}...")
cv_results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Let's check the type and structure of cv_results
print("Type of cv_results:", type(cv_results))
print("Keys in cv_results:", cv_results.keys())

# Try accessing the results differently
for key in cv_results:
    print(f"Key: {key}, Type: {type(cv_results[key])}")
    if hasattr(cv_results[key], 'mean'):
        print(f"Mean {key}:", cv_results[key].mean())
    else:
        print(f"Value {key}:", cv_results[key])


Evaluating NMF...
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9430  0.9459  0.9428  0.9461  0.9472  0.9450  0.0018  
MAE (testset)     0.7173  0.7187  0.7170  0.7188  0.7199  0.7183  0.0011  
Fit time          58.66   58.54   61.70   75.01   64.71   63.72   6.08    
Test time         3.88    3.57    4.45    4.37    3.72    4.00    0.35    
Type of cv_results: <class 'dict'>
Keys in cv_results: dict_keys(['test_rmse', 'test_mae', 'fit_time', 'test_time'])
Key: test_rmse, Type: <class 'numpy.ndarray'>
Mean test_rmse: 0.9450117363388463
Key: test_mae, Type: <class 'numpy.ndarray'>
Mean test_mae: 0.7183499573462457
Key: fit_time, Type: <class 'tuple'>
Value fit_time: (58.65761923789978, 58.538286447525024, 61.69825291633606, 75.01252555847168, 64.7076563835144)
Key: test_time, Type: <class 'tuple'>
Value test_time: (3.8787639141082764, 3.565955877304077, 4.453284978866577, 4.3660924434

# NMF Algorithm Results

| Metric | Fold 1 | Fold 2 | Fold 3 | Fold 4 | Fold 5 | Mean | Std |
|--------|--------|--------|--------|--------|--------|------|------|
| RMSE (testset) | 0.9430 | 0.9459 | 0.9428 | 0.9461 | 0.9472 | 0.9450 | 0.0018 |
| MAE (testset) | 0.7173 | 0.7187 | 0.7170 | 0.7188 | 0.7199 | 0.7183 | 0.0011 |
| Fit time | 58.66 | 58.54 | 61.70 | 75.01 | 64.71 | 63.72 | 6.08 |
| Test time | 3.88 | 3.57 | 4.45 | 4.37 | 3.72 | 4.00 | 0.35 |

We got a better result with NMF than SVD but it took longer to fit the model.

In [20]:
# Perform cross-validation for NMF algorithm
name = 'KNNBasic'
algorithm = KNNBasic(k=5, min_k=1, sim_options={'user_based': False})
print(f"\nEvaluating {name}...")
# due to performance constraints I had to take a smaller sample of the data
ratings_df_small = ratings_df.sample(frac=0.01, random_state=42)
data_small = Dataset.load_from_df(ratings_df_small[['userId', 'movieId', 'rating']], reader)

cv_results = cross_validate(algorithm, data_small, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Let's check the type and structure of cv_results
print("Type of cv_results:", type(cv_results))
print("Keys in cv_results:", cv_results.keys())

# Try accessing the results differently
for key in cv_results:
    print(f"Key: {key}, Type: {type(cv_results[key])}")
    if hasattr(cv_results[key], 'mean'):
        print(f"Mean {key}:", cv_results[key].mean())
    else:
        print(f"Value {key}:", cv_results[key])


Evaluating KNNBasic...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0581  1.0530  1.0681  1.0645  1.0564  1.0600  0.0055  
MAE (testset)     0.8375  0.8362  0.8495  0.8379  0.8381  0.8398  0.0049  
Fit time          0.48    0.34    0.22    0.19    0.19    0.28    0.11    
Test time         0.04    0.02    0.03    0.03    0.03    0.03    0.01    
Type of cv_results: <class 'dict'>
Keys in cv_results: dict_keys(['test_rmse', 'test_mae', 'fit_time', 'test_time'])
Key: test_rmse, Type: <class 'numpy.ndarray'>
Mean test_rmse

# KNN Algorithm Results

| Metric | Fold 1 | Fold 2 | Fold 3 | Fold 4 | Fold 5 | Mean | Std |
|--------|--------|--------|--------|--------|--------|------|------|
| RMSE (testset) | 1.0581 | 1.0530 | 1.0681 | 1.0645 | 1.0564 | 1.0600 | 0.0055 |
| MAE (testset) | 0.8375 | 0.8362 | 0.8495 | 0.8379 | 0.8381 | 0.8398 | 0.0049 |
| Fit time | 0.48 | 0.34 | 0.22 | 0.19 | 0.19 | 0.28 | 0.11 |
| Test time | 0.04 | 0.02 | 0.03 | 0.03 | 0.03 | 0.03 |

KNN with a 5 fold gets a decent result MAE 0.8398, but is very resource intensive I had to reduce the sample size by 100X for it to work with my laptop. 0.01 |

In [4]:
# Perform cross-validation for NMF algorithm
name = 'SlopeOne'
algorithm = SlopeOne()
print(f"\nEvaluating {name}...")
# due to performance constraints I had to take a smaller sample of the data
ratings_df_small = ratings_df.sample(frac=0.01, random_state=42)
data_small = Dataset.load_from_df(ratings_df_small[['userId', 'movieId', 'rating']], reader)

cv_results = cross_validate(algorithm, data_small, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Let's check the type and structure of cv_results
print("Type of cv_results:", type(cv_results))
print("Keys in cv_results:", cv_results.keys())

# Try accessing the results differently
for key in cv_results:
    print(f"Key: {key}, Type: {type(cv_results[key])}")
    if hasattr(cv_results[key], 'mean'):
        print(f"Mean {key}:", cv_results[key].mean())
    else:
        print(f"Value {key}:", cv_results[key])


Evaluating SlopeOne...
Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1313  1.1220  1.1010  1.1111  1.1292  1.1189  0.0114  
MAE (testset)     0.8839  0.8727  0.8578  0.8630  0.8789  0.8713  0.0097  
Fit time          0.74    0.73    0.55    0.56    0.59    0.63    0.08    
Test time         0.03    0.04    0.02    0.02    0.02    0.03    0.01    
Type of cv_results: <class 'dict'>
Keys in cv_results: dict_keys(['test_rmse', 'test_mae', 'fit_time', 'test_time'])
Key: test_rmse, Type: <class 'numpy.ndarray'>
Mean test_rmse: 1.1188960572588067
Key: test_mae, Type: <class 'numpy.ndarray'>
Mean test_mae: 0.8712760526240453
Key: fit_time, Type: <class 'tuple'>
Value fit_time: (0.7404730319976807, 0.7308573722839355, 0.548473596572876, 0.5568242073059082, 0.5934646129608154)
Key: test_time, Type: <class 'tuple'>
Value test_time: (0.027343273162841797, 0.037239789962768555, 0.0198707580

# SlopeOne Algorithm Results 

| Metric | Fold 1 | Fold 2 | Fold 3 | Fold 4 | Fold 5 | Mean | Std |
|--------|--------|--------|--------|--------|--------|------|------|
| RMSE (testset) | 1.1313 | 1.1220 | 1.1010 | 1.1111 | 1.1292 | 1.1189 | 0.0114 |
| MAE (testset) | 0.8839 | 0.8727 | 0.8578 | 0.8630 | 0.8789 | 0.8713 | 0.0097 |
| Fit time | 0.74 | 0.73 | 0.55 | 0.56 | 0.59 | 0.63 | 0.08 |
| Test time | 0.03 | 0.04 | 0.02 | 0.02 | 0.02 | 0.03 | 0.01 |

In [6]:
# Perform cross-validation for NMF algorithm
name = 'CoClustering'
algorithm = CoClustering()
print(f"\nEvaluating {name}...")
# due to performance constraints I had to take a smaller sample of the data
ratings_df_small = ratings_df.sample(frac=0.01, random_state=42)
data_small = Dataset.load_from_df(ratings_df_small[['userId', 'movieId', 'rating']], reader)

cv_results = cross_validate(algorithm, data_small, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Let's check the type and structure of cv_results
print("Type of cv_results:", type(cv_results))
print("Keys in cv_results:", cv_results.keys())

# Try accessing the results differently
for key in cv_results:
    print(f"Key: {key}, Type: {type(cv_results[key])}")
    if hasattr(cv_results[key], 'mean'):
        print(f"Mean {key}:", cv_results[key].mean())
    else:
        print(f"Value {key}:", cv_results[key])


Evaluating CoClustering...
Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1320  1.1468  1.1499  1.1256  1.1517  1.1412  0.0104  
MAE (testset)     0.8873  0.8986  0.9062  0.8847  0.9047  0.8963  0.0088  
Fit time          1.27    1.23    1.28    1.13    1.30    1.24    0.06    
Test time         0.01    0.01    0.07    0.01    0.07    0.04    0.03    
Type of cv_results: <class 'dict'>
Keys in cv_results: dict_keys(['test_rmse', 'test_mae', 'fit_time', 'test_time'])
Key: test_rmse, Type: <class 'numpy.ndarray'>
Mean test_rmse: 1.1411957991625283
Key: test_mae, Type: <class 'numpy.ndarray'>
Mean test_mae: 0.896310336858496
Key: fit_time, Type: <class 'tuple'>
Value fit_time: (1.270249843597412, 1.2317979335784912, 1.2778737545013428, 1.1315102577209473, 1.295992136001587)
Key: test_time, Type: <class 'tuple'>
Value test_time: (0.012739419937133789, 0.01221776008605957, 0.07467

# MovieLens Recommendation Algorithms Comparison

| Algorithm | RMSE | MAE | Fit Time (s) | Test Time (s) |
|-----------|------|-----|--------------|--------------|
| SVD | 0.8965 | 0.6851 | 41.98 | 5.78 |
| NMF | 0.9450 | 0.7183 | 63.72 | 4.00 |
| SlopeOne | 1.1189 | 0.8713 | 0.63 | 0.03 |
| CoClustering | 1.1412 | 0.8963 | 1.24 | 0.04 |
| KNNBasic | 1.06000 |0.83980 | 0.28 | 0.03 |

## Performance Analysis

1. **Best Accuracy**: SVD achieves the lowest RMSE (0.8965) and MAE (0.6851), making it the most accurate algorithm.

2. **Speed-Accuracy Tradeoff**:
   - SVD: Best accuracy but slower training (42s)
   - NMF: Good accuracy but slowest training (64s)
   - KNNBasic: Moderate accuracy but performance issues (had to reduce sample size)
   - SlopeOne: Less accurate but extremely fast (0.63s)
   - CoClustering: Least accurate but very fast (1.24s)

3. **Memory Requirements**:
   - KNNBasic failed due to memory constraints
   - The other algorithms completed successfully

## Recommendation

For the MovieLens dataset with 32M ratings, SVD provides the best prediction accuracy. If real-time updates or limited computational resources are a concern, SlopeOne offers a reasonable compromise with significantly faster training times.

In [ ]:
# Display the results
results_df = pd.DataFrame(results).T
print("\nResults:")
print(results_df.sort_values('test_rmse'))

# Plot the RMSE for each algorithm
plt.figure(figsize=(10, 6))
sns.barplot(x=results_df.index, y='test_rmse', data=results_df)
plt.title('RMSE by Algorithm')
plt.ylabel('RMSE (lower is better)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()